In [1]:
import sys
import os
import numpy as np
import zipfile
from tqdm import tqdm
import scrapbook as sb
from tempfile import TemporaryDirectory
# import tensorflow as tf

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources 
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.nrms import NRMS
from recommenders.models.newsrec.io.mind_iterator import MINDIterator
from recommenders.models.newsrec.io.mind_all_iterator import MINDAllIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set

import torch
import torch.nn as nn
import torch.nn.functional as F

print("System version: {}".format(sys.version))
# print("Tensorflow version: {}".format(tf.__version__))

System version: 3.8.16 (default, Mar  2 2023, 03:18:16) [MSC v.1916 64 bit (AMD64)]


In [2]:
print(1)

1


In [3]:
news_title = [""]
news_title.append(['hi'])
news_title

['', ['hi']]

In [4]:
import re
def word_tokenize(sent):
    """Split sentence into word list using regex.
    Args:
        sent (str): Input sentence
                                                                                                                                                                    
    Return:
        list: word list
    """
    pat = re.compile(r"[\w]+|[.,!?;|]")
    if isinstance(sent, str):
        return pat.findall(sent.lower())
    else:
        return []
word_tokenize('HI MY NAME IS')

['hi', 'my', 'name', 'is']

In [5]:
import numpy as np
a2d = np.array([[1, 2, 3], [4, 5, 6]])
a2d[[1] + [0]]

array([[4, 5, 6],
       [1, 2, 3]])

In [6]:
import numpy as np

In [60]:
epochs = 5
seed = 42
batch_size = 6

# Options: demo, small, large
MIND_type = 'demo'

In [61]:
import os

# Specify the name of the directory
directory = "recommenders\data"

# Specify the current path
current_path = os.getcwd()

# Full path
data_path = os.path.join(current_path, directory)

# Create the new directory
if not os.path.exists(data_path):
    os.makedirs(data_path)

print(f"Data path is set to: {data_path}")


Data path is set to: C:\Users\Rija Farooqui\Desktop\news_recommendation\recommenders\recommenders\data


In [62]:
train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
vertDict_file = os.path.join(data_path, "utils", "vert_dict.pkl")
subvertDict_file = os.path.join(data_path, "utils", "subvert_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'naml.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

In [63]:
# import pickle

# # specify the path to your .pkl file
# file_path = wordEmb_file

# # open and read the pickle file
# with open(file_path, 'rb') as f:
#     data = pickle.load(f)

# # print or inspect the content of the pickle file
# print(data)


In [64]:
if not os.path.exists(train_news_file):
download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)
    
if not os.path.exists(valid_news_file):
download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)

100%|████████████████████████████████████████████████████████████████████████████| 17.0k/17.0k [00:04<00:00, 4.04kKB/s]
100%|████████████████████████████████████████████████████████████████████████████| 9.84k/9.84k [00:03<00:00, 2.93kKB/s]


In [65]:
yaml_file

'C:\\Users\\Rija Farooqui\\Desktop\\news_recommendation\\recommenders\\recommenders\\data\\utils\\naml.yaml'

In [66]:
hparams = prepare_hparams(yaml_file, 
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file, 
                          userDict_file=userDict_file,
                          vertDict_file=vertDict_file,
                          subvertDict_file=subvertDict_file,
                          batch_size=batch_size,
                          epochs=epochs,
                          show_step=10)
print(hparams)

HParams object with values {'support_quick_scoring': True, 'dropout': 0.2, 'attention_hidden_dim': 200, 'head_num': 4, 'head_dim': 100, 'filter_num': 400, 'window_size': 3, 'vert_emb_dim': 100, 'subvert_emb_dim': 384, 'gru_unit': 400, 'type': 'ini', 'user_emb_dim': 50, 'learning_rate': 0.0001, 'optimizer': 'adam', 'epochs': 5, 'batch_size': 6, 'show_step': 10, 'title_size': 30, 'body_size': 50, 'his_size': 50, 'vert_num': 17, 'subvert_num': 249, 'cold_his_size': 10, 'data_format': 'naml', 'npratio': 4, 'metrics': ['group_auc', 'mean_mrr', 'ndcg@5;10'], 'embedding_type': 'bert-base-uncased', 'word_emb_dim': 768, 'cnn_activation': 'relu', 'model_type': 'naml', 'dense_activation': 'relu', 'loss': 'cross_entropy_loss', 'wordEmb_file': 'C:\\Users\\Rija Farooqui\\Desktop\\news_recommendation\\recommenders\\recommenders\\data\\utils\\embedding.npy', 'wordDict_file': 'C:\\Users\\Rija Farooqui\\Desktop\\news_recommendation\\recommenders\\recommenders\\data\\utils\\word_dict.pkl', 'userDict_file

In [67]:
hparams.metrics

['group_auc', 'mean_mrr', 'ndcg@5;10']

In [68]:
import pickle
def load_dict(file_path):
    """load pickle file

    Args:
        file path (str): file path

    Returns:
        object: pickle loaded object
    """
    with open(file_path, "rb") as f:
        return pickle.load(f)
uid2index = load_dict(hparams.userDict_file)
word_dict = load_dict(hparams.wordDict_file)

In [69]:
iterator = MINDAllIterator
train_iterator = iterator(hparams, hparams.npratio, col_spliter="\t")
test_iterator = iterator(hparams, col_spliter="\t")
train_batches = train_iterator.load_data_from_file(train_news_file, train_behaviors_file)
test_batches = test_iterator.load_data_from_file(valid_news_file, valid_behaviors_file)

In [56]:
gen = train_iterator.load_data_from_file(train_news_file, train_behaviors_file)
count = 0
progress_bar = tqdm(test_batches, desc='Evaluate', ncols=100)
for b in progress_bar:
    count+=1
count
# demo 47323*6 = 283938
# small 456833*6 = 2740998
# 2347592*6 = 14085552

Evaluate: 2347592it [14:59, 2610.87it/s]


2347592

In [70]:
hparams.head_dim  = 15

In [71]:
def _init_embedding(file_path):
    """Load pre-trained embeddings as a constant tensor.

    Args:
        file_path (str): the pre-trained glove embeddings file path.

    Returns:
        numpy.ndarray: A constant numpy array.
    """

    return np.load(file_path)


word2vec_embedding = _init_embedding(hparams.wordEmb_file)
word2vec_embedding = torch.tensor(word2vec_embedding)
criterion = nn.CrossEntropyLoss()  # or any other suitable loss function
model = NRMS(hparams, word2vec_embedding)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [72]:
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=hparams.learning_rate)
device

device(type='cuda')

In [77]:
from transformers import BertModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer(['[CLS] hello my name is [SEP]', 'qazi saad'])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'input_ids': [[101, 101, 7592, 2026, 2171, 2003, 102, 102], [101, 1053, 16103, 7842, 4215, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [74]:
x= ['s', ['hi', 'my']]
x = [' '.join(n) for n in x]
x = np.array(x)
np.asarray(x[[1] + [0]])

array(['hi my', 's'], dtype='<U5')

In [ ]:
arr = [[['i', 'am'], ['a', 'good']], [['i', 'am'], ['a', 'good']]]


In [22]:
%%time
model.fit_model(train_iterator, train_news_file, train_behaviors_file, optimizer)

Epoch 1: 0it [00:00, ?it/s]

embeddings torch.Size([300, 30, 768]) <class 'torch.Tensor'>


Epoch 1: 1it [00:05,  5.59s/it, Loss=1.61, Accuracy=0.733]

embeddings torch.Size([30, 30, 768]) <class 'torch.Tensor'>
embeddings torch.Size([300, 30, 768]) <class 'torch.Tensor'>


Epoch 1: 2it [00:06,  2.82s/it, Loss=1.61, Accuracy=0.65] 

...

embeddings torch.Size([30, 30, 768]) <class 'torch.Tensor'>
embeddings torch.Size([300, 30, 768]) <class 'torch.Tensor'>


Epoch 1: 127it [02:01,  1.09it/s, Loss=1.6, Accuracy=0.648]

embeddings torch.Size([30, 30, 768]) <class 'torch.Tensor'>
embeddings torch.Size([300, 30, 768]) <class 'torch.Tensor'>


Epoch 1: 127it [02:02,  1.04it/s, Loss=1.6, Accuracy=0.648]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ D:\Anaconda\envs\py36\lib\site-packages\IPython\core\magics\execution.py:1317 in time            │
│                                                                                                  │
│   1314 │   │   if mode=='eval':                                                                  │
│   1315 │   │   │   st = clock2()                                                                 │
│   1316 │   │   │   try:                                                                          │
│ ❱ 1317 │   │   │   │   out = eval(code, glob, local_ns)                                          │
│   1318 │   │   │   except:                                                                       │
│   1319 │   │   │   │   self.shell.showtraceback()                                                │
│   1320 │   │   │   │   return                                                                    │
│ <timed eval>:1 in <module>                                                                       │
│                                                                                                  │
│ C:\Users\Rija                                                                                    │
│ Farooqui\Desktop\news_recommendation\recommenders\recommenders\models\newsrec\models\nrms.py:173 │
│ in fit_model                                                                                     │
│                                                                                                  │
│   170 │   │   │   │   optimizer.zero_grad()                                                      │
│   171 │   │   │   │                                                                              │
│   172 │   │   │   │   # Forward pass                                                             │
│ ❱ 173 │   │   │   │   step_preds = self.forward(inputs)                                          │
│   174 │   │   │   │   # print('\nPredictions:', step_preds.shape, step_preds)                    │
│   175 │   │   │   │                                                                              │
│   176 │   │   │   │   # Calculate loss                                                           │
│                                                                                                  │
│ C:\Users\Rija                                                                                    │
│ Farooqui\Desktop\news_recommendation\recommenders\recommenders\models\newsrec\models\nrms.py:87  │
│ in forward                                                                                       │
│                                                                                                  │
│    84 │   │   clicks = clicks.reshape(num_click_docs * num_user, -1)                             │
│    85 │   │   cands = cands.reshape(num_cand_docs * num_user, -1)                                │
│    86 │   │   click_embed = self.doc_encoder(clicks)                                             │
│ ❱  87 │   │   cand_embed = self.doc_encoder(cands)                                               │
│    88 │   │   # seq_len = cand_embed.shape[2]                                                    │
│    89 │   │   click_embed = click_embed.reshape(num_user, num_click_docs, -1)                    │
│    90 │   │   cand_embed = cand_embed.reshape(num_user, num_cand_docs, -1)                       │
│                                                                                                  │
│ D:\Anaconda\envs\py36\lib\site-packages\torch\nn\modules\module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _gl

In [23]:
 %%time
_, labels, preds = model.evaluate_model(test_iterator, valid_news_file, valid_behaviors_file)

Evaluate: 0it [00:01, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ D:\Anaconda\envs\py36\lib\site-packages\IPython\core\magics\execution.py:1325 in time            │
│                                                                                                  │
│   1322 │   │   else:                                                                             │
│   1323 │   │   │   st = clock2()                                                                 │
│   1324 │   │   │   try:                                                                          │
│ ❱ 1325 │   │   │   │   exec(code, glob, local_ns)                                                │
│   1326 │   │   │   │   out=None                                                                  │
│   1327 │   │   │   │   # multi-line %%time case                                                  │
│   1328 │   │   │   │   if expr_val is not None:                                                  │
│ <timed exec>:1 in <module>                                                                       │
│                                                                                                  │
│ C:\Users\Rija                                                                                    │
│ Farooqui\Desktop\news_recommendation\recommenders\recommenders\models\newsrec\models\nrms.py:224 │
│ in evaluate_model                                                                                │
│                                                                                                  │
│   221 │     with torch.no_grad():                                                                │
│   222 │   │     for batch in progress_bar:                                                       │
│   223 │   │   │     inputs, step_labels = self._get_input_label_from_iter(batch)                 │
│ ❱ 224 │   │   │     inputs = [torch.tensor(inp).to(device) for inp in inputs]                    │
│   225 │   │   │     step_labels = torch.tensor(step_labels).float().to(device)                   │
│   226 │   │   │                                                                                  │
│   227 │   │   │     # forward pass                                                               │
│                                                                                                  │
│ C:\Users\Rija                                                                                    │
│ Farooqui\Desktop\news_recommendation\recommenders\recommenders\models\newsrec\models\nrms.py:224 │
│ in <listcomp>                                                                                    │
│                                                                                                  │
│   221 │     with torch.no_grad():                                                                │
│   222 │   │     for batch in progress_bar:                                                       │
│   223 │   │   │     inputs, step_labels = self._get_input_label_from_iter(batch)                 │
│ ❱ 224 │   │   │     inputs = [torch.tensor(inp).to(device) for inp in inputs]                    │
│   225 │   │   │     step_labels = torch.tensor(step_labels).float().to(device)                   │
│   226 │   │   │                                                                                  │
│   227 │   │   │     # forward pass                                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: can't convert np.ndarray of type numpy.str_. The only supported types are: float64, float32, float16, 
complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [24]:
from recommenders.models.deeprec.deeprec_utils import cal_metric
res = cal_metric(labels, preds, hparams.metrics)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\Rija Farooqui\AppData\Local\Temp\ipykernel_25880\3619223950.py:2 in <module>            │
│                                                                                                  │
│ [Errno 2] No such file or directory: 'C:\\Users\\Rija                                            │
│ Farooqui\\AppData\\Local\\Temp\\ipykernel_25880\\3619223950.py'                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'labels' is not defined

In [ ]:
res

In [ ]:
from elasticsearch import Elasticsearch

# Connect to Elasticsearch
es = Elasticsearch("http://localhost:9200")

# Define the analyzer/tokenizer configuration
tokenizer_config = {
    "tokenizer": "tokenizer_name",
    "text": "Sample text to analyze"
}

# Use the analyze API to inspect the tokenizer
analysis_result = es.indices.analyze(index='your_index_name', body=tokenizer_config)

# Print the tokenization result
for token in analysis_result['tokens']:
    print(token['token'])


In [ ]:
from elasticsearch import Elasticsearch

# Connect to Elasticsearch
es = Elasticsearch("http://localhost:9200")

# Define the index name
index_name = 'sudachi_sample'

# Define the Sudachi tokenizer and analyzer configuration
sudachi_config = {
    "settings": {
        "index": {
            "analysis": {
                "tokenizer": {
                    "sudachi_tokenizer": {
                        "type": "sudachi_tokenizer",
                        "split_mode": "C",
                        "discard_punctuation": True,
                        "resources_path": "sudachi"
                    }
                },
                "analyzer": {
                    "sudachi_analyzer": {
                        "filter": [],
                        "tokenizer": "sudachi_tokenizer",
                        "type": "custom"
                    }
                }
            }
        }
    }
}

# Create the index with the Sudachi tokenizer and analyzer configuration
# es.indices.create(index=index_name, body=sudachi_config)

# Test the analyzer by analyzing a sample text
sample_text = "日本の選挙管理委員会はとても良い"
analysis_result = es.indices.analyze(index=index_name, analyzer="sudachi_analyzer", text=sample_text)

# Print the tokens produced by the analyzer
tokens = [token["token"] for token in analysis_result["tokens"]]
print("Tokens:", tokens)